# Set Up

In [1]:
import pandas as pd

In [2]:
index_torgo = pd.read_csv('index_TORGO.txt', sep = '|')
index_torgo_filtered = index_torgo[(~index_torgo['prompt'].str.contains(']'))
                                   & (index_torgo['speaker'].isin(['F01',
                                                                   'F03',
                                                                   'F04',
                                                                   'M01',
                                                                   'M02',
                                                                   'M03',
                                                                   'M04',
                                                                   'M05']))
                                   & (~index_torgo['prompt'].str.contains('/'))]
index_torgo_filtered['url'] =   'https://s3.amazonaws.com/capstone-voice-for-all/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'
index_torgo_filtered['filepath'] = '/home/jovyan/work/data/TORGO/' \
                                + index_torgo_filtered['speaker'] + '/' \
                                + index_torgo_filtered['session'] + '/' \
                                + index_torgo_filtered['mic'] + '/' \
                                + index_torgo_filtered['prompt_id'] + '.wav'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
pd.options.display.max_colwidth = 200
index_torgo_filtered_top10 = index_torgo_filtered[['prompt', 'url', 'filepath']].sample(10)
index_torgo_filtered_top10

,prompt,url,filepath
1706,stubble,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F04/Session1/wav_arrayMic/0014.wav,/home/jovyan/work/data/TORGO/F04/Session1/wav_arrayMic/0014.wav
643,mere,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F03/Session1/wav_arrayMic/0172.wav,/home/jovyan/work/data/TORGO/F03/Session1/wav_arrayMic/0172.wav
1220,None,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F03/Session2/wav_arrayMic/0395.wav,/home/jovyan/work/data/TORGO/F03/Session2/wav_arrayMic/0395.wav
5714,torn,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/M05/Session1/wav_arrayMic/0058.wav,/home/jovyan/work/data/TORGO/M05/Session1/wav_arrayMic/0058.wav
1265,None,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F03/Session2/wav_arrayMic/0440.wav,/home/jovyan/work/data/TORGO/F03/Session2/wav_arrayMic/0440.wav
4934,train,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/M04/Session1/wav_arrayMic/0127.wav,/home/jovyan/work/data/TORGO/M04/Session1/wav_arrayMic/0127.wav
1177,None,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/F03/Session2/wav_arrayMic/0352.wav,/home/jovyan/work/data/TORGO/F03/Session2/wav_arrayMic/0352.wav
5853,I tried to tell people in the community.,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/M05/Session2/wav_headMic/0068.wav,/home/jovyan/work/data/TORGO/M05/Session2/wav_headMic/0068.wav
3306,drop,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/M02/Session1/wav_headMic/0144.wav,/home/jovyan/work/data/TORGO/M02/Session1/wav_headMic/0144.wav
5490,sleep,https://s3.amazonaws.com/capstone-voice-for-all/TORGO/M04/Session2/wav_arrayMic/0260.wav,/home/jovyan/work/data/TORGO/M04/Session2/wav_arrayMic/0260.wav


## Amazon Transcribe

[Info about Amazon Transcribe](https://aws.amazon.com/transcribe/)

In [4]:
from __future__ import print_function
from urllib.request import urlopen
import time
import boto3
import json

In [5]:
def amazon_translate(row):
    jobName = str(row.name)
    jobUri = row.url
    transcribe = boto3.client('transcribe')
    try:
        transcribe.delete_transcription_job(TranscriptionJobName=jobName)
    except:
        pass
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=jobName,
            Media={'MediaFileUri': jobUri},
            MediaFormat='wav',
            LanguageCode='en-US'
        )
    except:
        pass
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=jobName)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        # print(jobName + ' not ready yet...')
        time.sleep(5)
    result_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    with urlopen(result_url) as conn:
        transcript_html = conn.read()
        transcript = json.loads(transcript_html)
        transcript_text = transcript['results']['transcripts'][0]['transcript']
        transcribe.delete_transcription_job(TranscriptionJobName=jobName)
        print(jobName + ': ' + transcript_text)
        return transcript_text

In [ ]:
index_torgo_filtered_top10['awstranslate'] = index_torgo_filtered_top10.apply(amazon_translate, axis=1)
index_torgo_filtered_top10[['prompt', 'awstranslate']]

## Google Cloud Speech-to-Text

[Info about Speech-to-Text](https://cloud.google.com/speech-to-text/)

In [ ]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

In [ ]:
def google_translate(row):
    jobName = str(row.name)
    file_name = row.filepath
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='en-US')

    # Detects speech in the audio file
    response = client.recognize(config, audio)
    try:
        transcript = response.results[0].alternatives[0].transcript
        return transcript
    except:
        return None

In [ ]:
index_torgo_filtered_top10

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jovyan/work/capstone-c60aec7ead62.json"
client = speech.SpeechClient()

# index_torgo_filtered_top10['googletranslate'] = index_torgo_filtered_top10.apply(google_translate, axis=1)
# index_torgo_filtered_top10[['prompt', 'googletranslate']]

# Create loop to generate translations for google translate and aws

In [12]:
import os.path

index_torgo_loop_set = index_torgo_filtered

### Google Cloud Speech-to-Text

In [ ]:
while index_torgo_loop_set['url'].count() > 0:
    try:
        if os.path.isfile('google_translate.csv'):
            current_list = pd.read_csv('google_translate.csv', index_col = 0)
            index_torgo_loop_set = index_torgo_loop_set[~index_torgo_loop_set['url'].isin(current_list['url'])]
            if os.path.isfile('google_failed.csv'):
                failed_list = pd.read_csv('google_failed.csv', index_col = 0)
                index_torgo_loop_set = index_torgo_loop_set[~index_torgo_loop_set['url'].isin(failed_list['url'])]
            
        print("Records left to translate: %s"%(index_torgo_loop_set['url'].count()))

        index_torgo_loop_set_sample = index_torgo_loop_set.sample(1)
        index_torgo_loop_set_sample['googletranslate'] = index_torgo_loop_set_sample.apply(google_translate, axis=1)

        if os.path.isfile('google_translate.csv'):
            export_list = pd.concat([current_list, index_torgo_loop_set_sample])
            export_list.to_csv('google_translate.csv')
        else:
            index_torgo_loop_set_sample.to_csv('google_translate.csv')
    except BaseException as e:
        print('Processing failed with exception: '+str(e))
        if os.path.isfile('google_failed.csv'):
            export_failed_list = pd.concat([failed_list, index_torgo_loop_set_sample])
            export_failed_list.to_csv('google_failed.csv')
        else:
            index_torgo_loop_set_sample.to_csv('google_failed.csv')
            
            

### Amazon Transcribe

In [13]:
while index_torgo_loop_set['url'].count() > 1:
    try:
        if os.path.isfile('aws_translate.csv'):
            current_list = pd.read_csv('aws_translate.csv', index_col = 0)
            index_torgo_loop_set = index_torgo_loop_set[~index_torgo_loop_set['url'].isin(current_list['url'])]
            if os.path.isfile('aws_failed.csv'):
                failed_list = pd.read_csv('aws_failed.csv', index_col = 0)
                index_torgo_loop_set = index_torgo_loop_set[~index_torgo_loop_set['url'].isin(failed_list['url'])]
            
        print("Records left to translate: %s"%(index_torgo_loop_set['url'].count()))

        index_torgo_loop_set_sample = index_torgo_loop_set.sample(1)
        index_torgo_loop_set_sample['awstranslate'] = index_torgo_loop_set_sample.apply(amazon_translate, axis=1)

        if os.path.isfile('aws_translate.csv'):
            export_list = pd.concat([current_list, index_torgo_loop_set_sample])
            export_list.to_csv('aws_translate.csv')
        else:
            index_torgo_loop_set_sample.to_csv('aws_translate.csv')
    except BaseException as e:
        print('Processing failed with exception: '+str(e))
        if os.path.isfile('aws_failed.csv'):
            export_failed_list = pd.concat([failed_list, index_torgo_loop_set_sample])
            export_failed_list.to_csv('aws_failed.csv')
        else:
            index_torgo_loop_set_sample.to_csv('aws_failed.csv')
            
            

Records left to translate: 30
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6116')
Records left to translate: 29
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6123')
Records left to translate: 28
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6122')
Records left to translate: 27
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6134')
Records left to translate: 26
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6143')
Records left to translate: 25
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6138')
Records left to translate: 24
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6137')
Records left to translate: 23
Processing failed with exception: ('TranscriptFileUri', 'occurred at index 6133')
Records left to translate: 22
Processing failed with exception: ('TranscriptFileUri', 'occurred at index

In [ ]:
index_torgo_filtered_top10[['prompt', 'googletranslate', 'awstranslate']]